In [27]:
import glob,os
import pandas as pd
import numpy as np

In [10]:
import sys
sys.path.append(os.path.expanduser("~"))
from ss2csv.ss2csv import file2table,cleancols

In [11]:
GDIR = os.path.expanduser("~") + "/" + "Dropbox/lml/genome_2021/"

In [12]:
allfiles = glob.glob(os.path.join(GDIR,"*.txt"))

In [22]:
badfiles = []
for i,allfile in enumerate(allfiles):
    try:
        tmp = cleancols(file2table(allfile))
    except:
        badfiles.append(allfile)
        continue
        
    if i==0:
        df = tmp
    else:
        try:
            df = pd.concat([df,tmp])
        except:
            badfiles.append(allfile)

bad row
bad row


#### badfiles

In [28]:
with open(badfiles[0],'r') as colorfile:
    l = colorfile.readlines()
        
    hunt = False
    collist = []
    for item in l:
        if hunt==True:
            if 'END_DATA_FORMAT' in item:
                break
            else:
                collist.append(item)

        if 'BEGIN_DATA_FORMAT' in item:
            hunt = True    
    
    cols = collist[0].split('\t')
    cols = [item.rstrip('\n') for item in cols]
    sf = pd.DataFrame(columns=cols)
        
    hunt = False
    vallists = []
    for item in l:
        if hunt==True:
            if item=='END_DATA\n':
                vallists.append(vallist)
                hunt = False
            else:
                vallist.append(item)

        if item=='BEGIN_DATA\n':
            hunt = True
            vallist = []
            
megalist = vallists[0]
l = [item.rstrip('\n').split('\t') for item in megalist]
l = [item for sublist in l for item in sublist]
l = [item for item in l if item!='']

# turn megalist into Series whose indices are rows of sf
megalist_length = len(l)
ncols = len(sf.columns)
nrows = int( megalist_length / ncols )
valseries_idx = list(np.repeat(range(nrows),ncols))
valseries = pd.Series(l,index=valseries_idx)

for i in range(nrows):
    sf.loc[i] = list(valseries.loc[i]) # making it a list wipes its index, which we need

In [32]:
df = pd.concat([df,cleancols(sf)])

In [33]:
df = df.reset_index(drop=True)

In [35]:
df.to_csv('table.csv',index=False)